In [35]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING")
os.environ["LANGSMITH_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")

In [36]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

In [37]:
response = llm.invoke("Hello! how are you?")

print(response.content)

I am doing well, thank you for asking!  How are you today?


In [38]:
PDF_TO_TEXT_PROMPT = """
You are a highly accurate document parser. Your task is to extract *everything* from a PDF document.

The input is raw PDF content (converted to text and image snippets). Extract and preserve the following:

1. **All textual content**, including:
   - Page numbers
   - Headers and footers
   - Paragraphs, tables, lists, and titles
   - Questions and answers (if present)
2. **Image content** should be included as summaries or captions like:
   - [Image]: A diagram showing a neural network architecture.
3. Preserve the **page structure** with clear delimiters like:
   - === Page 1 ===
   - === Page 2 ===

Make sure your output is:
- In Markdown
- Preserves the order and layout as much as possible
- Suitable for semantic chunking and retrieval-based systems

Do not skip any content. Your goal is to represent the document *completely and accurately* for storage in a vector database.

Now extract the content.
"""

In [21]:
# from langchain_community.document_loaders import PyPDFLoader

# loader = PyPDFLoader("resources/HSC26-Bangla1st-Paper-4.pdf")
# documents = loader.load()

In [44]:
import base64

with open("resources/HSC26-Bangla1st-Paper-4.pdf", "rb") as f:
    pdf_bytes = f.read()
    pdf_base64 = base64.b64encode(pdf_bytes).decode()

In [41]:
messages = [
    SystemMessage(content=PDF_TO_TEXT_PROMPT),
    HumanMessage(content=[
        {
            "type": "text",
            "text": "Please extract and process all text from this PDF document, paying special attention to Bangla/Bengali content."
        },
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:application/pdf;base64,{pdf_base64}"
            }
        }
    ])
]

In [45]:
# import time

# start_time = time.time()
# response = llm.invoke(messages)
# processing_time = time.time() - start_time


# print(processing_time)

In [46]:
# from langchain_core.messages import SystemMessage, HumanMessage

# extracted_texts = []
# for doc in documents:
#     prompt = [
#         SystemMessage(content=PDF_TO_TEXT_PROMPT),
#         HumanMessage(doc.page_content)
#     ]
#     response = llm.invoke(prompt)
#     extracted_texts.append(response.content)

# # Combine extracted texts
# bangla_text = "\n".join(extracted_texts)

In [24]:
from IPython.display import display
display(bangla_text[:500])

'শব্দার্ব ও টীকা\nেূ  শব্দ শলব্দি অর্ব ও িযাখ্যা\nমনু-সংহীতা র্ব্ধানকতিা ব্া িাস্ত্রপ্রকণতা মুর্নর্ব্কিষ।\nমনু-সংহীতা মনু-প্রণীত মানুকষি আচিণর্ব্র্ধ সংক্রান্ত গ্রন্থ ।\nপ্রিাপর্ত িীকব্ি রষ্ট্া। ব্রহ্মা। ইর্ন র্ব্ক ি যদব্তা।\nপঞ্চিি মদনকদকব্ি ব্যব্হােি পঁাচ ধিকনি ব্াণ।\nকন্সটি নানা িকম ব্াদযেকন্ত্রি ঐকতান।\nযসকিা স্বণিকাি, যসানাি অলংকাি প্রস্তুতকািক\nব্ব্িি যকালাহকলি মি হস্তী দ্বািা \nসংগীতসিস্বতীি পদ্মব্ন\nদজলত র্ব্দজলত কর্ি া আর্ম যতা \nর্ব্ব্াহ-ব্ার়্েকত র্গ া\nউঠিলাম\nঅনুপম র্নকিি র্ব্ব্াহোত্রাি পর্িজির্ত'

## Google SDK

In [52]:
import google.generativeai as genai

gemini_model = genai.GenerativeModel('gemini-1.5-flash')

In [53]:
uploaded_file = genai.upload_file("resources/HSC26-Bangla1st-Paper-4.pdf", mime_type="application/pdf")
        
        
start_time = time.time()
response = gemini_model.generate_content([PDF_TO_TEXT_PROMPT, uploaded_file])
processing_time = time.time() - start_time
        
genai.delete_file(uploaded_file.name)

print(processing_time)

12.135200262069702


In [55]:
extracted_text = response.text

print(f"Total character = {len(extracted_text)}")

Total character = 1564


In [57]:
from IPython.display import display, Markdown

display(Markdown(extracted_text))

=== Page 1 ===

**শব্দার্থ ও টীকা**

| মূল শব্দ | শব্দের অর্থ ও ব্যাখ্যা |
|---|---|
| মনু-সংহীতা | বিধানকর্তা বা শাস্ত্রপ্রণেতা মুনিবিশেষ। |
| মনু-সংহীতা | মনু-প্রণীত মানুষের আচরণবিধি সংক্রান্ত গ্রন্থ। |
| প্রজাপতি | জীবের স্রষ্টা। ব্রহ্মা। ইনি বিয়ের দেবতা। |
| পঞ্চশর | মদনদেবের ব্যবহার্য পাঁচ ধরনের বাণ। |
| কন্সর্ট | নানা রকম বাদ্যযন্ত্রের ঐকতান। |
| সেকরা | স্বর্ণকার, সোনার অলংকার প্রস্তুতকারক |
| বর্বর কোলাহলের মত্ত হস্তী দ্বারা <br> সংগীতসরস্বতীর পদ্মবন <br> দলিত বিদলিত করিয়া আমি তো <br> বিবাহ-বাড়িতে গিয়া <br> উঠিলাম | অনুপম নিজের বিবাহযাত্রার পরিস্থিতি বর্ণনায় সুরশূন্য <br> বিকট কোলাহলকে সংগীত সরস্বতীর পদ্মবন দলিত হওয়ার <br> ঘটনার সঙ্গে তুলনা করেছে। |
| অভিষিক্ত | অভিষেক করা হয়েছে এমন |
| সওগাঁদ | উপঢৌকন। ভেট। |
| দেওয়া-থোওয়া | যে পাথরে ঘষে সোনার খাঁটিত্ব পরীক্ষা করা হয় |
| কষ্টিপাথর | আলবোলা। ফরসি। দীর্ঘ নলযুক্ত হুকাবিশেষ |
| মকরমুখো মোটা একখানা বালা | মকর বা কুমিরের মুখাকৃতিযুক্ত হাতে পরিধেয় <br> অলংকারবিশেষ |
| এয়ারিং | কানের দুল। Earring |
| দক্ষযজ্ঞ | প্রজাপতি দক্ষ কর্তৃক অনুষ্ঠিত যজ্ঞ। এ যজ্ঞে পতিনিন্দা শুনে <br> সতী দেহত্যাগ করেন। স্ত্রীর মৃত্যুসংবাদ শুনে শিব <br> অনুচরসহ যজ্ঞস্থলে পৌছে যজ্ঞ ধ্বংস করে দেন এবং সতীর <br> শব কাঁধে তুলে নিয়ে প্রলয় নৃত্যে মত্ত হন। এখানে প্রলয়কাণ্ড <br> বা হট্টগোল বোঝাচ্ছে। |
| রসনচৌকি | শানাই, ঢোল ও কাঁসি- এই তিন বাদ্যযন্ত্রে সৃষ্ট ঐকতানবাদন |
| অভ্র | এক ধরনের খনিজ ধাতু। Mica |
| অভ্রের ঝাড় | অভ্রের তৈরি ঝাড়বাতি। |
| মহানির্বাণ | সবরকমের বন্ধন থেকে মুক্তি। |
| কলি | পুরাণে বর্ণিত চার যুগের শেষ যুগ। কলিযুগ। |
| কলি যে চারপোয়া হইয়া আসিল। | কলিকাল পরিপূর্ণরূপে আত্মপ্রকাশ করল। |

4
